In [1]:
import pandas as pd
from sla_calculation import (
    load_business_calendar,
    check_sla_compliance,
)

In [2]:
silver_issues_path = "../../data/silver/silver_issues.parquet"
silver_calendar_path = "../../data/silver/silver_calendar.parquet"

df = pd.read_parquet(silver_issues_path)
calendar_df = load_business_calendar(silver_calendar_path)

In [3]:
df = df[df["status"].isin(["Done", "Resolved"])].copy()
df.head()

,issue_id,issue_type,status,priority,assignee_email,assignee_id,assignee_name,created_at,resolved_at
1,JIRA-0002,Bug,Resolved,High,francinne@fasttrack.com,u004,Francinne,2025-03-09 12:39:26+00:00,2025-03-10 10:39:26+00:00
3,JIRA-0004,Story,Done,High,francinne@fasttrack.com,u004,Francinne,2025-11-22 20:24:58+00:00,2025-11-23 07:24:58+00:00
4,JIRA-0005,Task,Resolved,Medium,francinne@fasttrack.com,u004,Francinne,2025-01-06 01:08:44+00:00,2025-01-07 09:08:44+00:00
7,JIRA-0008,Story,Resolved,High,matheus.malta@fasttrack.com,u001,Matheus Malta,2025-04-07 22:55:41+00:00,2025-04-08 14:55:41+00:00
8,JIRA-0009,Story,Resolved,Medium,gionni.lucio@fasttrack.com,u002,Gionni Lucio,2025-10-08 21:21:55+00:00,2025-10-12 10:21:55+00:00


In [4]:
sla_results = []

for _, row in df.iterrows():
    result = check_sla_compliance(
        created_at=row["created_at"],
        resolved_at=row["resolved_at"],
        priority=row["priority"],
        calendar_df=calendar_df,
    )
    sla_results.append(result)

sla_df = pd.DataFrame(sla_results)

df = pd.concat([df.reset_index(drop=True), sla_df], axis=1)

df.head()

,issue_id,issue_type,status,priority,assignee_email,assignee_id,assignee_name,created_at,resolved_at,resolution_hours,sla_expected_hours,is_sla_met
0,JIRA-0002,Bug,Resolved,High,francinne@fasttrack.com,u004,Francinne,2025-03-09 12:39:26+00:00,2025-03-10 10:39:26+00:00,2.66,24,True
1,JIRA-0004,Story,Done,High,francinne@fasttrack.com,u004,Francinne,2025-11-22 20:24:58+00:00,2025-11-23 07:24:58+00:00,0.00,24,True
2,JIRA-0005,Task,Resolved,Medium,francinne@fasttrack.com,u004,Francinne,2025-01-06 01:08:44+00:00,2025-01-07 09:08:44+00:00,11.15,72,True
3,JIRA-0008,Story,Resolved,High,matheus.malta@fasttrack.com,u001,Matheus Malta,2025-04-07 22:55:41+00:00,2025-04-08 14:55:41+00:00,6.93,24,True
4,JIRA-0009,Story,Resolved,Medium,gionni.lucio@fasttrack.com,u002,Gionni Lucio,2025-10-08 21:21:55+00:00,2025-10-12 10:21:55+00:00,20.00,72,True


In [7]:
gold_sla_issues = df[
    [
        "issue_id",
        "issue_type",
        "assignee_name",
        "priority",
        "created_at",
        "resolved_at",
        "resolution_hours",
        "sla_expected_hours",
        "is_sla_met",
    ]
]
gold_sla_issues.head()

,issue_id,issue_type,assignee_name,priority,created_at,resolved_at,resolution_hours,sla_expected_hours,is_sla_met
0,JIRA-0002,Bug,Francinne,High,2025-03-09 12:39:26+00:00,2025-03-10 10:39:26+00:00,2.66,24,True
1,JIRA-0004,Story,Francinne,High,2025-11-22 20:24:58+00:00,2025-11-23 07:24:58+00:00,0.00,24,True
2,JIRA-0005,Task,Francinne,Medium,2025-01-06 01:08:44+00:00,2025-01-07 09:08:44+00:00,11.15,72,True
3,JIRA-0008,Story,Matheus Malta,High,2025-04-07 22:55:41+00:00,2025-04-08 14:55:41+00:00,6.93,24,True
4,JIRA-0009,Story,Gionni Lucio,Medium,2025-10-08 21:21:55+00:00,2025-10-12 10:21:55+00:00,20.00,72,True


In [10]:
gold_sla_by_analyst = (
    gold_sla_issues
    .groupby("assignee_name")
    .agg(
        issue_count=("issue_id", "count"),
        avg_sla_hours=("resolution_hours", "mean"),
    )
    .reset_index()
)
gold_sla_by_analyst.head()

,assignee_name,issue_count,avg_sla_hours
0,Adyna,148,15.604527
1,Francinne,175,15.992057
2,Gionni Lucio,175,16.343029
3,Guilherme Francisco,157,16.180000
4,Matheus Malta,149,14.536443


In [11]:
gold_sla_by_issue_type = (
    gold_sla_issues
    .groupby("issue_type")
    .agg(
        issue_count=("issue_id", "count"),
        avg_sla_hours=("resolution_hours", "mean"),
    )
    .reset_index()
)
gold_sla_by_issue_type.head()

,issue_type,issue_count,avg_sla_hours
0,Bug,237,15.875316
1,Story,290,15.046759
2,Task,277,16.419819


In [12]:
gold_sla_issues.to_csv("../../data/gold/gold_sla_issues.csv",index=False,)
gold_sla_by_analyst.to_csv("../../data/gold/gold_sla_by_analyst.csv", index=False,)
gold_sla_by_issue_type.to_csv("../../data/gold/gold_sla_by_issue_type.csv",index=False,)